<a href="https://colab.research.google.com/github/elifsare/Titanic-Survival-Data/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [195]:
from google.colab import drive
drive.mount('/content/drive')
import os
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  MyDrive	output.csv  sample_data  weights.best.hdf5


In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [197]:
def preprocess(data):
  data.Cabin.fillna('0', inplace=True)
  data.loc[data.Cabin.str[0] == 'A', 'Cabin'] = 0
  data.loc[data.Cabin.str[0] == 'B', 'Cabin'] = 1
  data.loc[data.Cabin.str[0] == 'C', 'Cabin'] = 2
  data.loc[data.Cabin.str[0] == 'D', 'Cabin'] = 3
  data.loc[data.Cabin.str[0] == 'E', 'Cabin'] = 4
  data.loc[data.Cabin.str[0] == 'F', 'Cabin'] = 5
  data.loc[data.Cabin.str[0] == 'G', 'Cabin'] = 6
  data.loc[data.Cabin.str[0] == 'T', 'Cabin'] = 7
    
  # Cinsiyeti tam sayıya çevirelim
  data['Sex'].replace('female', 0, inplace=True)
  data['Sex'].replace('male', 1, inplace=True)
    
  # Gemiye biniş limanlarını tam sayıya çevirelim
  data['Embarked'].replace('S', 0, inplace=True)
  data['Embarked'].replace('C', 1, inplace=True)
  data['Embarked'].replace('Q', 2, inplace=True)
    
  # Olmayan (NA) yaş değerlerini medyan ile dolduralım
  data['Age'].fillna(data['Age'].median(), inplace=True)
  data['Fare'].fillna(data['Fare'].median(), inplace=True)
  data['Embarked'].fillna(data['Embarked'].median(), inplace=True)
    
  # İstersek olmayan (NA) değerleri tamamen silebiliriz
  # data.dropna(subset=['Fare', 'Embarked'], inplace=True, how='any')
  return data



In [198]:
# data içindeki gupların başlıklarıni değiştirme
def group_titles(data):
    data['Names'] = data['Name'].map(lambda x: len(re.split(' ', x)))
    data['Title'] = data['Name'].map(lambda x: re.search(', (.+?) ', x).group(1))
    data['Title'].replace('Master.', 0, inplace=True)
    data['Title'].replace('Mr.', 1, inplace=True)
    data['Title'].replace(['Ms.','Mlle.', 'Miss.'], 2, inplace=True)
    data['Title'].replace(['Mme.', 'Mrs.'], 3, inplace=True)
    data['Title'].replace(['Dona.', 'Lady.', 'the Countess.', 'Capt.', 'Col.', 'Don.', 'Dr.', 'Major.', 'Rev.', 'Sir.', 'Jonkheer.', 'the'], 4, inplace=True)

In [199]:
def data_subset(data):
    features = ['Pclass', 'SibSp', 'Parch', 'Sex', 'Names', 'Title', 'Age', 'Cabin'] #, 'Fare', 'Embarked']
    lengh_features = len(features)
    subset = data[features]#.fillna(0)
    return subset, lengh_features


In [200]:
# modelin içinde neler olacak, tasarım
# Define Sequential model with 3 layers
# Also, let's build a Sequential model incrementally via the add() method
def create_model(train_set_size, input_length, num_epochs, batch_size):
  model = Sequential()
  model.add(Dense(7, input_dim=input_length, activation='softplus'))
  model.add(Dense(3, activation='softplus'))
  model.add(Dense(1, activation='softplus'))

  lr = .001 # Learning_rate, Defaults to 0.001.
  adam0 = Adam(lr = lr)

  # loss='binary_crossentropy'
  # Entropy is a measure of the uncertainty associated with a given distribution q(y).
  # Binary crossentropy is a loss function that is used in binary classification tasks. These are tasks that answer a question with only two choices. 
  #Several independent such questions can be answered at the same time, as in multi-label classification.
  # Configures the model for training --> model.compile

  model.compile(loss='binary_crossentropy', optimizer=adam0, metrics=['accuracy']) # metrik olarak doğruluk ölçümü

  filepath = 'weights.best.hdf5' # öğrenilen ağırlığın kaydı için dosya

  checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max') # checkpoint yardımıyla best result'ları belirler
  callbacks_list = [checkpoint]
  
  history_model = model.fit(X_train[:train_set_size], Y_train[:train_set_size], callbacks=callbacks_list, epochs=num_epochs, batch_size=batch_size, verbose=0) #40, 32
  return model, history_model

In [201]:
def plots(history):
  loss_history = history.history['loss']
  acc_history = history.history['accuracy']
  epochs = [(i + 1) for i in range(num_epochs)]

  ax = plt.subplot(211)
  ax.plot(epochs, loss_history, color='red')
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Error Rate\n')
  ax.set_title('Error Rate per Epoch\n')

  ax2 = plt.subplot(212)
  ax2.plot(epochs, acc_history, color='blue')
  ax2.set_xlabel('Epochs')
  ax2.set_ylabel('Accuracy\n')
  ax2.set_title('Accuracy per Epoch\n')

  plt.subplots_adjust(hspace=0.8)
  plt.savefig('Accuracy_loss.png')
  plt.close()                                 

In [202]:
def test(batch_size):
    
    test = pd.read_csv('/content/drive/MyDrive/Titanic/test.csv', header=0)
    test_ids = test['PassengerId']
    test = preprocess(test)
    group_titles(test)
    testdata, _ = data_subset(test)

    X_test = np.array(testdata).astype(float)

    output = model.predict(X_test, batch_size=batch_size, verbose=0)
    output = output.reshape((418,))

    column_0 = np.concatenate((['PassengerId'], test_ids ), axis=0 )
    column_1 = np.concatenate( ( ['Survived'], output ), axis=0 )

    f = open("output.csv", "w")
    writer = csv.writer(f)
    for i in range(len(column_0)):
        writer.writerow( [column_0[i]] + [column_1[i]])
    f.close()

In [203]:
# fonksiyonları da çağırarak programın bütün olacağı kısım

In [204]:
seed = 7
np.random.seed(seed)

train = pd.read_csv('/content/drive/MyDrive/Titanic/train.csv', header=0)

preprocess(train)
group_titles(train)

num_epochs = 100
batch_size = 32

train_data, lengh_features = data_subset(train)

Y_train = np.array(train['Survived']).astype(int)
X_train = np.array(train_data).astype(float)


train_set_size = int(.67 * len(X_train))


model, history_model = create_model(train_set_size, lengh_features, num_epochs, batch_size)

plots(history_model)

X_validation = X_train[train_set_size:]
Y_validation = Y_train[train_set_size:]

loss_and_metrics = model.evaluate(X_validation, Y_validation, batch_size=batch_size)
print ("loss_and_metrics")

test(batch_size)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: accuracy improved from -inf to 0.60738, saving model to weights.best.hdf5

Epoch 2: accuracy did not improve from 0.60738

Epoch 3: accuracy did not improve from 0.60738

Epoch 4: accuracy did not improve from 0.60738

Epoch 5: accuracy did not improve from 0.60738

Epoch 6: accuracy did not improve from 0.60738

Epoch 7: accuracy did not improve from 0.60738

Epoch 8: accuracy did not improve from 0.60738

Epoch 9: accuracy did not improve from 0.60738

Epoch 10: accuracy improved from 0.60738 to 0.61577, saving model to weights.best.hdf5

Epoch 11: accuracy did not improve from 0.61577

Epoch 12: accuracy improved from 0.61577 to 0.62248, saving model to weights.best.hdf5

Epoch 13: accuracy improved from 0.62248 to 0.62416, saving model to weights.best.hdf5

Epoch 14: accuracy improved from 0.62416 to 0.63591, saving model to weights.best.hdf5

Epoch 15: accuracy improved from 0.63591 to 0.64262, saving model to weights.best.hdf5

Epoch 16: accuracy improved from 0.64262 t